In [1]:
# pip install mediapipe

In [7]:
import os
import pandas as pd
import mediapipe as mp
import cv2
import math
import numpy as np

In [8]:
path = '../32_imgs/'
os.chdir(path)
uploaded = os.listdir(path)
uploaded.sort()

In [8]:
# 이미지의 원하는 높이와 너비 (480*480)
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

# 이미지를 받아서 크기를 조절하고 표시하는 역할을 함.
def resize(image):

  # 입력 이미지의 높이와 너비를 가져옴
  h, w = image.shape[:2]

  # 이미지의 높이가 너비보다 작을 경우
  if h < w:
    # 이미지의 너비를 'DESIRED_WIDTH로 조절하고, 높이는 비율에 따라 조절
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  # 아닌 경우
  else:
    # 이미지의 높이를 'DESIRED_WIDTH'로 조절하고, 너비는 비율에 따라 조절
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))

# Read images with OpenCV.
images = {name: cv2.imread(name) for name in uploaded}

# resize 함수를 통해 크기 조절
for name, image in images.items():
  resize(image)

#### 좌표값 -> 데이터프레임

In [9]:
mp_pose = mp.solutions.pose # 포즈 추정 모델을 사용할 수 있게 하는 인터페이스 제공, 포즈 감지 & 랜드마크 탐색
mp_drawing = mp.solutions.drawing_utils # 포즈 추정 결과를 시각적으로 표시
mp_drawing_styles = mp.solutions.drawing_styles

# 결과값의 랜드마크 이름 목록
landmark_names = [
    "NOSE", "LEFT_EYE_INNER", "LEFT_EYE", "LEFT_EYE_OUTER", "RIGHT_EYE_INNER",
    "RIGHT_EYE", "RIGHT_EYE_OUTER", "LEFT_EAR", "RIGHT_EAR", "MOUTH_LEFT",
    "MOUTH_RIGHT", "LEFT_SHOULDER", "RIGHT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW",
    "LEFT_WRIST", "RIGHT_WRIST", "LEFT_PINKY", "RIGHT_PINKY", "LEFT_INDEX",
    "RIGHT_INDEX", "LEFT_THUMB", "RIGHT_THUMB", "LEFT_HIP", "RIGHT_HIP", "LEFT_KNEE",
    "RIGHT_KNEE", "LEFT_ANKLE", "RIGHT_ANKLE", "LEFT_HEEL", "RIGHT_HEEL",
    "LEFT_FOOT_INDEX", "RIGHT_FOOT_INDEX"
]

# columns 명 설정
columns = []
for landmark_name in landmark_names:
    columns.extend([f"{landmark_name}.x", f"{landmark_name}.y", f"{landmark_name}.z"])

df = pd.DataFrame(columns=columns)

#  MediaPipe Pose 모델 초기화, Pose 모델 객체 생성
with mp_pose.Pose(
    # 정적 이미지 처리, 최소 검출 신뢰도 0.5, 모델의 복잡성 중간 수준
    static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:

  # images라는 딕셔너리에서 각각의 이미지와 해당 이미지의 이름 가져옴
  for name, image in images.items():
    # 이미지의 색공간을 BGR에서 RGB로 변환, pose.process() 함수를 사용하여 Pose모델에 이미지를 전달하여 포즈 추정
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # 결과값을 저장할 리스트 초기화
    landmark_data = []

    # 각 랜드마크의 좌표값을 landmark_data 리스트에 추가
    for landmark in results.pose_world_landmarks.landmark:
        landmark_data.extend([landmark.x, landmark.y, landmark.z])

    # 데이터프레임에 행 추가
    # df = df.append(pd.DataFrame([landmark_data], columns=columns), ignore_index=True)
    df = pd.concat([df, pd.DataFrame([landmark_data], columns=columns)], axis = 0)


# 데이터프레임을 CSV 파일로 저장
#df.to_csv('/gdrive/MyDrive/Colab_Notebooks/push_up_test.csv', index=False)

In [10]:
df

,NOSE.x,NOSE.y,NOSE.z,LEFT_EYE_INNER.x,LEFT_EYE_INNER.y,LEFT_EYE_INNER.z,LEFT_EYE.x,LEFT_EYE.y,LEFT_EYE.z,LEFT_EYE_OUTER.x,...,LEFT_HEEL.z,RIGHT_HEEL.x,RIGHT_HEEL.y,RIGHT_HEEL.z,LEFT_FOOT_INDEX.x,LEFT_FOOT_INDEX.y,LEFT_FOOT_INDEX.z,RIGHT_FOOT_INDEX.x,RIGHT_FOOT_INDEX.y,RIGHT_FOOT_INDEX.z
0,-0.702182,0.227786,-0.221773,-0.698043,0.182756,-0.256874,-0.696796,0.183373,-0.244538,-0.699692,...,0.340137,0.687330,-0.045382,0.449268,0.698174,0.128187,0.342121,0.658810,0.055955,0.457947
0,-0.703927,0.217500,-0.213131,-0.698983,0.177674,-0.251274,-0.698247,0.178337,-0.239271,-0.700945,...,0.360513,0.685635,-0.041311,0.468474,0.694670,0.117056,0.360068,0.654019,0.066473,0.481507
0,-0.717482,0.065400,-0.260763,-0.705923,0.023150,-0.294051,-0.705222,0.023627,-0.281755,-0.707266,...,0.370205,0.672575,0.046248,0.460782,0.658774,0.215724,0.369251,0.618375,0.135947,0.459900
0,-0.663301,-0.052807,-0.252938,-0.658139,-0.094704,-0.289764,-0.657528,-0.093832,-0.276104,-0.658900,...,0.322727,0.684662,0.183016,0.401798,0.637763,0.356499,0.295220,0.602943,0.255440,0.366972
0,-0.666801,-0.035984,-0.250369,-0.661418,-0.078048,-0.287141,-0.661226,-0.077095,-0.273440,-0.662420,...,0.342703,0.675830,0.188082,0.421247,0.637134,0.346558,0.316052,0.596454,0.258890,0.388727
0,-0.701477,0.099094,-0.273636,-0.692517,0.064472,-0.307872,-0.691170,0.065005,-0.296144,-0.693895,...,0.410051,0.706443,0.001444,0.497580,0.661493,0.182277,0.409459,0.656722,0.101632,0.493892
0,-0.691649,0.189716,-0.241503,-0.686399,0.152078,-0.279709,-0.685349,0.153002,-0.267851,-0.688133,...,0.410613,0.686076,-0.047941,0.511457,0.681876,0.118237,0.410871,0.648706,0.056086,0.523811
0,-0.687055,0.198467,-0.237588,-0.683873,0.163881,-0.274647,-0.683045,0.164673,-0.263106,-0.685607,...,0.422117,0.684960,-0.026182,0.513466,0.687890,0.114014,0.424264,0.649548,0.079699,0.526365
0,-0.691853,0.210848,-0.225617,-0.688920,0.173485,-0.263458,-0.688091,0.174187,-0.251760,-0.690664,...,0.368643,0.682324,-0.029326,0.481540,0.688250,0.123106,0.368365,0.646281,0.078305,0.494634
0,-0.691777,0.211036,-0.225385,-0.688102,0.172660,-0.262844,-0.687267,0.173389,-0.251081,-0.689843,...,0.377535,0.668122,-0.017740,0.487025,0.680813,0.132257,0.379446,0.632123,0.089680,0.501799


In [2]:
def resize(image, DESIRED_WIDTH, DESIRED_HEIGHT):

  # 입력 이미지의 높이와 너비를 가져옴
  h, w = image.shape[:2]

  # 이미지의 높이가 너비보다 작을 경우
  if h < w:
    # 이미지의 너비를 'DESIRED_WIDTH로 조절하고, 높이는 비율에 따라 조절
    img = cv2.resize(image, (DESIRED_WIDTH, math.floor(h/(w/DESIRED_WIDTH))))
  # 아닌 경우
  else:
    # 이미지의 높이를 'DESIRED_WIDTH'로 조절하고, 너비는 비율에 따라 조절
    img = cv2.resize(image, (math.floor(w/(h/DESIRED_HEIGHT)), DESIRED_HEIGHT))

In [3]:
def img2data(img_div_path, data_div_path):         # 작동완료
   
    # img_div_path = '../32_imgs/'
    # data_div_path = '../data'

    path = img_div_path
    os.chdir(path)
    uploaded = os.listdir(path)
    uploaded.sort()

    # 이미지의 원하는 높이와 너비 (480*480)
    DESIRED_HEIGHT = 480
    DESIRED_WIDTH = 480

    # Read images with OpenCV.
    images = {name: cv2.imread(name) for name in uploaded}

    # resize 함수를 통해 크기 조절
    for name, image in images.items():
      resize(image, DESIRED_WIDTH, DESIRED_HEIGHT)

    mp_pose = mp.solutions.pose # 포즈 추정 모델을 사용할 수 있게 하는 인터페이스 제공, 포즈 감지 & 랜드마크 탐색
    mp_drawing = mp.solutions.drawing_utils # 포즈 추정 결과를 시각적으로 표시
    mp_drawing_styles = mp.solutions.drawing_styles

    # 결과값의 랜드마크 이름 목록
    landmark_names = [
        "NOSE", "LEFT_EYE_INNER", "LEFT_EYE", "LEFT_EYE_OUTER", "RIGHT_EYE_INNER",
        "RIGHT_EYE", "RIGHT_EYE_OUTER", "LEFT_EAR", "RIGHT_EAR", "MOUTH_LEFT",
        "MOUTH_RIGHT", "LEFT_SHOULDER", "RIGHT_SHOULDER", "LEFT_ELBOW", "RIGHT_ELBOW",
        "LEFT_WRIST", "RIGHT_WRIST", "LEFT_PINKY", "RIGHT_PINKY", "LEFT_INDEX",
        "RIGHT_INDEX", "LEFT_THUMB", "RIGHT_THUMB", "LEFT_HIP", "RIGHT_HIP", "LEFT_KNEE",
        "RIGHT_KNEE", "LEFT_ANKLE", "RIGHT_ANKLE", "LEFT_HEEL", "RIGHT_HEEL",
        "LEFT_FOOT_INDEX", "RIGHT_FOOT_INDEX"
    ]

    # columns 명 설정
    columns = []
    for landmark_name in landmark_names:
        columns.extend([f"{landmark_name}.x", f"{landmark_name}.y", f"{landmark_name}.z"])

    df = pd.DataFrame(columns=columns)

    #  MediaPipe Pose 모델 초기화, Pose 모델 객체 생성
    with mp_pose.Pose(
        # 정적 이미지 처리, 최소 검출 신뢰도 0.5, 모델의 복잡성 중간 수준
        static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:

      # images라는 딕셔너리에서 각각의 이미지와 해당 이미지의 이름 가져옴
      for name, image in images.items():
        # 이미지의 색공간을 BGR에서 RGB로 변환, pose.process() 함수를 사용하여 Pose모델에 이미지를 전달하여 포즈 추정
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # 결과값을 저장할 리스트 초기화
        landmark_data = []

        # 각 랜드마크의 좌표값을 landmark_data 리스트에 추가
        for landmark in results.pose_world_landmarks.landmark:
            landmark_data.extend([landmark.x, landmark.y, landmark.z])

        # 데이터프레임에 행 추가
        df = pd.concat([df, pd.DataFrame([landmark_data], columns=columns)], axis = 0)

        output_path = data_div_path + 'coordinate.csv'
        df.to_csv(output_path, index=False)

In [9]:
img2data('../32_imgs/', '../data/')